In [1]:
import numpy as np
from numba import jit

In [2]:
@jit(nopython=True)
def idx2site(Lx,Ly,idx):
    return np.vstack((idx%Lx,idx//Lx))

In [3]:
@jit(nopython=True)
def site2idx(Lx,Ly,x,y):
    return x+Lx*y

In [4]:
@jit(nopython=True)
def main_1():
    Lx = 4
    Ly = 3
    site = np.array([1])
    print("site --> idx",site,idx2site(Lx,Ly,site))
    idxs = np.arange(Lx*Ly)
    print("inds",idxs)
    sites = idx2site(Lx,Ly,idxs)
    print("sites",sites)
    print("xs",sites[0])
    print("ys",sites[1])
    idxs2 = site2idx(Lx,Ly,sites[0],sites[1])
    print("idxs2",idxs2)

In [5]:
main_1()

site --> idx [1] [[1]
 [0]]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
sites [[0 1 2 3 0 1 2 3 0 1 2 3]
 [0 0 0 0 1 1 1 1 2 2 2 2]]
xs [0 1 2 3 0 1 2 3 0 1 2 3]
ys [0 0 0 0 1 1 1 1 2 2 2 2]
idxs2 [ 0  1  2  3  4  5  6  7  8  9 10 11]


In [6]:
@jit(nopython=True)
def makeHam(L,Lhbond,i1s,i2s,Js):
    Ham = np.zeros((L,L),dtype=np.float64)
    for i in range(Lhbond):
        Ham[i1s[i],i2s[i]] = Js[i]
        Ham[i2s[i],i1s[i]] = Js[i]
    return Ham

In [7]:
@jit(nopython=True)
def main_2():
    L = 10
## periodic
#    i1s = np.arange(L)
#    i2s = (i1s + 1)%L
## open
    i1s = np.array([i for i in range(L-1)])
    i2s = np.array([(i+1)%L for i in range(L-1)])
    Js = - np.ones(L-1)
    Lhbond = len(Js)
    print("i1s",i1s)
    print("i2s",i2s)
    print("Js",Js)
    Ham = makeHam(L,Lhbond,i1s,i2s,Js)
    print(Ham)
    ene, vec = np.linalg.eigh(Ham)
    for i in range(L):
        vec[:,i] *= np.sign(vec[0,i])
    print(vec[:,0])
    print(vec[:,0].T@Ham@vec[:,0],ene[0])
#
    eps = np.array([-2.0 * np.cos((n+1) * np.pi / (L+1)) for n in range(L)])
    x = np.zeros((L,L), dtype = np.float64)
    for j in range(L):
        for k in range(L):
            x[j,k] = np.sin((j+1)*(k+1)*np.pi / (L+1))
    x *= np.sqrt(2.0 / (L+1))
    x = x.T
    print(x[:,0])
    print(x[:,0].T@Ham@x[:,0],eps[0])
## vec=x or not
    print("sum(|vec-x|)",np.sum(np.abs(vec-x)))

In [8]:
main_2()

i1s [0 1 2 3 4 5 6 7 8]
i2s [1 2 3 4 5 6 7 8 9]
Js [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
[[ 0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]]
[0.12013117 0.23053002 0.3222527  0.38786839 0.42206128 0.42206128
 0.38786839 0.3222527  0.23053002 0.12013117]
-1.9189859472289952 -1.9189859472289947
[0.12013117 0.23053002 0.3222527  0.38786839 0.42206128 0.42206128
 0.38786839 0.3222527  0.23053002 0.12013117]
-1.918985947228995 -1.9189859472289947
sum(|vec-x|) 2.67841304690819e-14


In [9]:
@jit(nopython=True)
def main_3():
    Lx = 4
    Ly = 3
## open
    i1s = []
    i2s = []
    Js = []
    for y in range(Ly):
        for x in range(Lx):
            i0 = site2idx(Lx,Ly,x,y)
            if x+1<Lx:
                ix = site2idx(Lx,Ly,x+1,y)
                i1s.append(i0)
                i2s.append(ix)
                Js.append(-1.0)
            if y+1<Ly:
                iy = site2idx(Lx,Ly,x,y+1)
                i1s.append(i0)
                i2s.append(iy)
                Js.append(-1.0)
    Lhbond = len(Js)
    print("i1s",i1s)
    print("i2s",i2s)
    print("Js",Js)
    print("Lhbond",Lhbond,2*Lx*Ly-Lx-Ly)
#
    L = Lx*Ly
    Ham = makeHam(L,Lhbond,i1s,i2s,Js)
    print(Ham)
    ene, vec = np.linalg.eigh(Ham)
    for i in range(L):
        vec[:,i] *= np.sign(vec[0,i])
    print(vec[:,0])
    print(vec[:,0].T@Ham@vec[:,0],ene[0])
    print(vec[:,1])
    print(vec[:,1].T@Ham@vec[:,1],ene[1])
#
    eps = np.array([-2.0 * (np.cos((i+1)*np.pi/(Lx+1)) + np.cos((j+1)*np.pi/(Ly+1))) for i in range(Lx) for j in range(Ly)])
    print(eps)

In [10]:
main_3()

i1s [0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6, 7, 8, 9, 10]
i2s [1, 4, 2, 5, 3, 6, 7, 5, 8, 6, 9, 7, 10, 11, 9, 10, 11]
Js [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
Lhbond 17 17
[[ 0. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0. -1.  0.  0. -1.  0.  0.  0.]
 [ 0. -1.  0.  0. -1.  0. -1.  0.  0. -1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  0. -1.  0.  0. -1.  0.]
 [ 0.  0.  0. -1.  0.  0. -1.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  0.]]
[0.18587402 0.30075048 0.30075048 0.18587402 0.26286556 0.4253254
 0.4253254  0.26286556 0.18587402 0.30075048 0.30075048 0.18587402]
-3.03224755112298